### Integración de Romberg

Para la integración de Romberg necesitamos:

- Un R<sub>nn</sub>: Donde "n" se utilizará en los siguientes pasos
- Una integral definida planteada adecuadamente

Método:

- Se identifica la función a integrar y los límites de integración
- A partir de los límites desde "a" a "b" determino mi "h" como "b-a"
- Se genera una matriz de ceros de orden (n, n)
- La primera fila de la matriz se llena con el uso de la regla compuesta del trapecio utilizando n = 2^i, donde i empieza en zero hasta n-1
- Saco el valor exacto y lo comparo con el último valor de los valores de la primera columna de la matriz, no debe ser demasiado diferente
- Lleno el resto de valores de la matriz con la versión espejo de la extrapolación de Richarson.

In [1]:
# Ejemplo de vídeo https://youtu.be/5jsUtiX8DOI?si=irHUXJBdd-buyssl

from sympy import *
from numpy import zeros, float64

x = symbols("x")

f = pi * ( (1+ sqrt(4 - x**2))**2 -1)

#
n = 5
a = -2
b = 2

# declaro mi matriz (n, n)
R = zeros((n, n), dtype=float64)


# Utilizo la función que he desarrollado para la regla compuesta del trapecio

def regla_compuesta_del_trapecio(f, a, b, n):
    # Defino mi h dentro de la función
    h = (b-a) / n
    # La suma interna es el factor de la fórmula, la inicio con f(x_0)
    suma_interna = f.subs(x, a)
    # Aquí hago la sumatoria desde f(x_1) hasta f(x_(n-1)) 
    for i in range(1, n):
        suma_interna += 2 * f.subs(x, a + i*h)
    # Sumo f(x_n)
    suma_interna += f.subs(x, b)
    return (h/2) * suma_interna

# Y encuentro cada valor con mi función donde "n" de la regla compuesta
# Se duplica en cada iteración

R11 = float(regla_compuesta_del_trapecio(f, a, b, 1))
R21 = float(regla_compuesta_del_trapecio(f, a, b, 2))
R31 = float(regla_compuesta_del_trapecio(f, a, b, 4))
R41 = float(regla_compuesta_del_trapecio(f, a, b, 8))
R51 = float(regla_compuesta_del_trapecio(f, a, b, 16))

# Ahora estos valor les pongo el la primera columna de mi matriz R

R[0][0] = R11
R[1][0] = R21
R[2][0] = R31
R[3][0] = R41
R[4][0] = R51

# Saco mi valor exacto a modo de verificación

valor_exacto = float(integrate(f, (x, a, b)))


# Ahora lleno el resto de la matriz con la versión espejo de la
# Interpolación de Richarson

for j in range(1, n):
    for i in range(j, n):
        # La j es diferente a la fórmula por la forma de indexar de Python
        R[i][j] = (4**(j) * R[i][j-1] - R[i-1][j-1]) / (4**(j) - 1)

#imprimo de forma que se vea bien la matriz
for i in R:
    for j in i:
        print(str(j).ljust(18), end="  ")
    print("\n")

# el valor R[n-1][n-1] es el valor aproximado

valor_aproximado = R[n-1][n-1]
error = abs(valor_aproximado - valor_exacto)
error_relativo = round(float(error/valor_exacto * 100), 4)

print("El valor exacto es:", valor_exacto)
print("El valor aproximado es:", valor_aproximado)
print("El error es:", error)
print(f"El error relativo es: {error_relativo}%")

0.0                 0.0                 0.0                 0.0                 0.0                 

50.26548245743669   67.02064327658225   0.0                 0.0                 0.0                 

65.74788952106772   70.9086918756114    71.16789511554667   0.0                 0.0                 

70.63191326526466   72.25992117999697   72.35000313362268   72.36876675295723   0.0                 

72.20723444372594   72.73234150321304   72.76383619142743   72.77040497012274   72.77198002195477   

El valor exacto es: 72.98873924264856
El valor aproximado es: 72.77198002195477
El error es: 0.21675922069378828
El error relativo es: 0.297%


### Integración por método de Cuadratura Gaussiana

Para integrar por cuadratura Gaussiana:

- Planteo mi integral con los límites establecidos. En métodos numéricos las constantes deben estar dentro de la integral.
- Hago el cambio de variable y de límites, para trasladar la integral a un intervalo [-1,1] con la fórmulas:

$$ x = \frac{1}{2} [(b-a)t + a + b] $$
$$ dx = \frac{1}{2} (b-a)dt $$

- Sustituyo "x" y "dx" (o la variable que corresponda) en la integral y cambio los límites a *a=-1*, *b=1*.
- Obtengo las raices del polinomio de legendre que nos diga el problema, para ello uno el *n* que se me proporcione. Saco estas raíces con el comando *nroots(pln)*.
- Obtengo los pesos con el bucle *for* que indico abajo. Este for tiene que llegar hasta *n*.
- Obtengo el valor aproximado con el bucle *for* como en el ejemplo abajo, este for tienen que llegar hasta *n*.
- obtengo el valor exacto en la función y límites originales y comparo.




In [2]:

pl2 = x**2 - (1/3)
pl3 = x**3 - (3/5)*x
pl4 = x**4 - (6/7)*x**2 + (3/35)
pl5 = x**5 - (10/9)*x**3 + (5/21)*x
pl6 = x**6 - (15/11)*x**4 + (5/11)*x**2 - (5/231)
pl7 = x**7 - (21/13)*x**5 + (105/143)*x**3 - (35/429)*x
pl8 = x**8 - (28/15)*x**6 + (14/13)*x**4 - (28/143)*x**2 + (7/1287)


raices = nroots(pl5)
print(raices)

#Derivada del polonomio 5
dp5 = diff(pl5)

# obtengo los pesos en una lista con el siguiente for
pesos = []
for i in range(5):
    pesos.append(float((1/ dp5.subs(x, raices[i])) * integrate(pl5 / (x-raices[i]), (x, -1, 1))))

print(pesos)

[-0.906179845938664, -0.538469310105683, 0, 0.538469310105683, 0.906179845938664]
[0.2369268850561892, 0.47862867049936686, 0.5688888888888886, 0.47862867049936675, 0.2369268850561892]


In [7]:
# Ejemplo de https://youtu.be/AcKbL3ttDDM?si=Ex56SMUSihavWvjj que utiliza 

# La integral planteada en el ejercicio es:

f = 2.5 * sqrt(1 + (0.5 * pi * cos(pi * x + pi))**2)

valor_aproximado = 0

for i in range(5):
    valor_aproximado += pesos[i]*f.subs(x, raices[i])

print(float(valor_aproximado))

# Obtengo el valor exacto en la función original
g = 25*sqrt(1 + (0.5* pi * cos(10*pi*x))**2)


valor_exacto = float(integrate(g, (x, 0, 0.2)))
print(valor_exacto)

# NOTA: El resultado da lo mismo independientemente del orden de las raices. 

7.222637820180375
7.318477362067681
